#### iLQR for kinematic example

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
from ocp import *
from costs import *
from ocp_utils import *

import pybullet as p
import pybullet_data

%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=4, suppress=True)

#### Setup pybullet with the urdf

In [2]:
# configure pybullet and load plane.urdf and quadcopter.urdf
#physicsClient = p.connect(p.DIRECT)  # pybullet only for computations no visualisation, faster
physicsClient = p.connect(p.GUI)  # pybullet with visualisation
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)

p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [3]:
p.resetSimulation()

robot_urdf = "../data/urdf/frankaemika_new/panda_arm.urdf"
robot_id = p.loadURDF(robot_urdf, useFixedBase=1)
joint_limits = get_joint_limits(robot_id, 7)

p.loadURDF('plane.urdf')

#Define the end-effector
link_id = 10
link_name = 'panda_grasptarget_hand'

#Create a ball to show the target
_,_,ballId = create_primitives(shapeType=p.GEOM_BOX, halfExtents=[0.07, 0.03, 0.02], rgbaColor=[1,0.4, 0,0.5])


#### Finding the joint (and link) index 

### Construct the robot system

In [4]:
dt = 0.05
T = 100
dof = 7
sys = URDFRobot(dof=dof, robot_id=robot_id, dt = dt)

#### Find a random target orientation 

In [5]:
qT = np.random.uniform(-1,1,7)
qT[3] -= 1.5
qT[5] += 1.5
xT = np.concatenate([qT, np.zeros(7)])
sys.set_init_state(xT)
_,target_orn = sys.compute_ee(xT,link_id)
print("Target orientation: ",target_orn)

Target orientation:  [0.7686 0.0416 0.3249 0.5495]


#### Set the initial state

In [6]:
q0 = np.random.rand(7)
q0 = np.array([0.,0.,0.,-1.5,0.,1.5,0.])
#q0 = np.array([0.4201, 0.4719, 0.9226, 0.8089, 0.3113, 0.7598, 0.364 ])
x0 = np.concatenate([q0, np.zeros(7)])
sys.set_init_state(x0)
_,x0_orn = sys.compute_ee(x0,link_id)
print("X0 orientation: ",x0_orn)

X0 orientation:  [ 0.9239  0.3827 -0.      0.    ]


#### Try forward kinematics

In [7]:
pos0, quat0 = sys.compute_ee(x0, link_id)

#Put the ball at the end-effector
p.resetBasePositionAndOrientation(ballId, pos0, quat0)
print(pos0, quat0)

[ 0.5477 -0.      0.5465] [ 0.9239  0.3827 -0.      0.    ]


#### Set reference

In [9]:
#### Define the object target and rotation
p_ref = np.array([0.7, -.3, 0.3])
angle = 0.4
quat_obj = np.array([0,0,np.sin(angle/2), np.cos(angle/2)])
R_obj = quat2Mat(quat_obj)
p.resetBasePositionAndOrientation(ballId, p_ref, quat_obj)

#W and WT: cost coefficients for the end-effector reaching task
W = np.eye(3)*1
WT = np.eye(3)*100

#Ignore the error at the idx-th axis in the object frame
# idx = 0
# W[idx] *= 0.0
# WT[idx] *= 0.0

p.resetBasePositionAndOrientation(ballId, pos0, quat_obj)

In [10]:
target_orn = np.array([np.sin(np.pi/2), 0,0,np.cos(np.pi/2)])

#### Set initial control output

In [11]:
#set initial control output to be all zeros
us = np.zeros((T+1,sys.Du))
_ = sys.compute_matrices(x0, us[0])
xs = sys.rollout(us[:-1])

#### Plot initial trajectory

#### Set the regularization cost coefficients Q and R 

In [12]:
Q = np.eye(sys.Dx)*1
Q[0:sys.dof,0:sys.dof] *= 0.01  #only put cost regularization on the velocity, not on the joint angles
Qf = np.eye(sys.Dx)*1
Qf[0:sys.dof,0:sys.dof] *= 0.01 #only put cost regularization on the velocity, not on the joint angles
R = np.eye(sys.Du)*1e-9
mu = 1e-9          #regularization coefficient

#### Set the task space cost coefficients W & WT

In [13]:
W = np.diag([1,1,1])
WT = np.diag([10,10,10])

### iLQR using cost model

#### Define the cost

In [14]:
#The costs consist of: a) state regularization (Q), b) control regularization (R), and c) End-effector reaching task (W)
#Running cost is for the time 0 <= t < T, while terminal cost is for the time t = T
runningStateCost = CostModelQuadratic(sys, Q)
runningControlCost = CostModelQuadratic(sys, None, R)
runningEECost = CostModelQuadraticOrientation(sys,W, link_id, o_ref = target_orn, R_ref=R_obj)
runningCost = CostModelSum(sys, [runningStateCost, runningControlCost, runningEECost])

terminalStateCost = CostModelQuadratic(sys,Qf)
terminalControlCost = CostModelQuadratic(sys, None,R)
terminalEECost = CostModelQuadraticOrientation(sys,WT, link_id,o_ref = target_orn, R_ref=R_obj)
terminalCost = CostModelSum(sys, [terminalStateCost, terminalControlCost, terminalEECost])

costs = [runningCost]*T + [terminalCost]

#### Construct ILQR

In [15]:
ilqr_cost = ILQR(sys, mu)
ilqr_cost.set_init_state(x0)
ilqr_cost.set_timestep(T)
ilqr_cost.set_cost(costs)
ilqr_cost.set_state(xs, us) #set initial trajectory

In [16]:
ilqr_cost.mu = mu

#### Solve and Plot

In [17]:
n_iter = 30
ilqr_cost.solve(n_iter, method='recursive')
xs_batch, us_batch = ilqr_cost.xs, ilqr_cost.us

#clear_output()

Cost converges at iteration 2, cannot decrease further


#### Play traj

In [18]:
sys.vis_traj(ilqr_cost.xs)

#### Compute Error

In [19]:
pos, quat = sys.compute_ee(ilqr_cost.xs[-1], link_id)

## Check the orientation derivative

Validate the derivative calculation of the orientation cost using numerical differentiation. This is a good practice when creating a new cost function, to check if our analytical calculation is correct. 

In [20]:
idx = np.random.randint(len(ilqr_cost.xs))
x = ilqr_cost.xs[idx]
u = ilqr_cost.us[idx]

In [21]:
inc = 0.001
J = np.zeros((1, len(x)))
for i in range(len(xs[0])):
    x_p = x.copy()
    x_p[i] += inc
    cost_p = runningEECost.calc(x_p, u)
    
    x_m = x.copy()
    x_m[i] -= inc
    cost_m = runningEECost.calc(x_m, u)
    
    diff = (cost_p-cost_m)/(2*inc)
    J[0,i] = diff
    
J_true = runningEECost.calcDiff(x, u)
J_true = runningEECost.Lx
print(J)
print(J_true)
print(J-J_true)

[[ 0.0783  0.0005  0.0783 -0.      0.0056  0.0003 -0.0783  0.      0.
   0.      0.      0.      0.      0.    ]]
[ 0.0783  0.0005  0.0783 -0.      0.0056  0.0003 -0.0783  0.      0.
  0.      0.      0.      0.      0.    ]
[[-0.  0.  0. -0. -0. -0.  0.  0.  0.  0.  0.  0.  0.  0.]]
